In [ ]:
# Import necessary libraries
import os
import subprocess

# Install required dependencies
!apt-get install -y build-essential
!apt-get install -y zlib1g-dev
!apt-get install -y python3-pip
!pip install pyrosetta

# Import PyRosetta
import pyrosetta
from pyrosetta import rosetta

# Initialize PyRosetta
pyrosetta.init()

# Define the sequences of the tRNA charging enzyme and the tRNA
enzyme_sequence = "YOUR_ENZYME_SEQUENCE_HERE"
trna_sequence = "cacgccctgtaggaattgaacccacgacatcaggttttgaaaacctgcgttctaccgactgaactaaggacgt"

# Create a pose object for the enzyme and the tRNA
enzyme_pose = rosetta.core.pose.Pose()
trna_pose = rosetta.core.pose.Pose()

# Create a function to generate a pose from a sequence
def pose_from_sequence(sequence):
    pose = rosetta.core.pose.Pose()
    rosetta.core.pose.make_pose_from_sequence(pose, sequence, rosetta.core.chemical.CENTROID)
    return pose

# Generate poses for the enzyme and the tRNA
enzyme_pose = pose_from_sequence(enzyme_sequence)
trna_pose = pose_from_sequence(trna_sequence)

# Create a combined pose by appending the tRNA pose to the enzyme pose
combined_pose = enzyme_pose.clone()
combined_pose.append_pose_by_jump(trna_pose, 1)

# Define a function to perform cofolding using Rosetta's docking protocol
def cofold(pose):
    # Create a docking protocol object
    docking_protocol = rosetta.protocols.docking.DockingProtocol()

    # Set docking options
    docking_protocol.set_partners("A_B") # Assuming the enzyme is chain A and the tRNA is chain B

    # Perform docking
    docking_protocol.apply(pose)

    return pose

# Perform cofolding
cofolded_pose = cofold(combined_pose)

# Save the resulting structure as a PDB file
output_pdb_file = "/content/cofolded_trna_enzyme.pdb"
cofolded_pose.dump_pdb(output_pdb_file)

print(f"Cofolded structure saved to {output_pdb_file}")
